# Introduction 
---
I this project we'll be working with Oily Giant mining company. Our task is to find the best place for a new well development. We'll build a model to predict the each region's well's reserves. Profit calculations with risk assessments will be conducted for development of 200 oil wells at a budget of 100 million USD. The best region for development will be suggested using our models and bootstapping technique.

The 3 datasets we'll be using can be found below:<ul>
    <a href='https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_0.csv'>Region 1</a>
    <a href='https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_1.csv'>Region 2</a>
    <a href='https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_2.csv'>Region 3</a>
    </ul><br>
The data consists of the following:<ul>
- id - well identification number<br>
- f0,f1,f2 - 3 features of point<br>
- product - volume of reserves in the well (thousand barrels)</ul>

The stages for this project will be as follows:<ol>
    1. Data Overview<br>
    2. Data Preprocessing<br>
    3. Model Creation/Testing<br>
    4. Profit Calculations<br>
    5. Risk Assessment<br>
    6. Top Region Selection/Conclusion

## Data Overview
---

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR

In [2]:
# Loading datasets
geo_0 = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_0.csv')
geo_1 = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_1.csv')
geo_2 = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_2.csv')

In [3]:
# Getting data overviews
geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


No missing values and all dtypes look good. Going to check t=for duplicates next.

In [6]:
print("geo_0 diuplicates:", geo_0.duplicated().sum())
print("geo_1 diuplicates:", geo_1.duplicated().sum())
print("geo_2 diuplicates:", geo_2.duplicated().sum())

geo_0 diuplicates: 0
geo_1 diuplicates: 0
geo_2 diuplicates: 0


No duplicates found

In [7]:
# Checking DataFrame content
print(f"geo_0:\n{geo_0[:3]}\n\ngeo_1:\n{geo_1[:3]}\n\ngeo_2:\n{geo_2[:3]}")

geo_0:
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647

geo_1:
      id         f0        f1        f2     product
0  kBEdx -15.001348 -8.276000 -0.005876    3.179103
1  62mP7  14.272088 -3.475083  0.999183   26.953261
2  vyE1P   6.263187 -5.948386  5.001160  134.766305

geo_2:
      id        f0        f1        f2    product
0  fwXo0 -1.146987  0.963328 -0.828965  27.758673
1  WJtFt  0.262778  0.269839 -2.530187  56.069697
2  ovLUW  0.194587  0.289035 -5.586433  62.871910


In [8]:
print("geo_0:\n",
      geo_0['product'].describe(),
      "\n\ngeo_1\n",
      geo_1['product'].describe(),
      "\n\ngeo_2:\n",
      geo_2['product'].describe()
)

geo_0:
 count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64 

geo_1
 count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64 

geo_2:
 count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64


Let's check for any outliers

In [9]:
# Checking for outliers
print("outliers in geo_0: ", (st.zscore(geo_0["product"])>3).sum())
print("outliers in geo_1: ", (st.zscore(geo_1["product"])>3).sum())
print("outliers in geo_2: ", (st.zscore(geo_2["product"])>3).sum())


outliers in geo_0:  0
outliers in geo_1:  0
outliers in geo_2:  0


No outliers found past 3 standard deviations of the averages within the respective datasets.

In [10]:
# Checking for correclations
print("geo_0 correlations:\n",geo_0[geo_0.columns[1:]].corr())
print("\ngeo_1 correlations:\n",geo_1[geo_1.columns[1:]].corr())
print("\ngeo_2 correlations:\n",geo_2[geo_2.columns[1:]].corr())

geo_0 correlations:
                f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

geo_1 correlations:
                f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

geo_2 correlations:
                f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


`f2` shows a positive correlation along with `f0` with a slight positive correlation for the first region's dataset. `f1` shows a negative correlation. Each feature demonstrates influence on the value of the product for each observation.

The second region's data is showing a much different correlation, indicating each region will need its own model as the previous region's model will have been trained on significantly different correlations.

`f2` has a heavily positive correlation whereas `f1` and `f0` don't show much correlation

The third region's `f0` and `f1` features don't show much correlation to the product value but `f2` shows a moderate positive correlation

We've got an average of 68.8 thousand barrels per welll int this region. The standard deviation is higher than region 1

## Model Creation/Testing
---

We'll be creating a linear regression model that predicts the amount each well's yield within each region. We'll create a model for each region, get the root mean squared error scores and analyze results.

In [11]:
# Creasting features and target variables for geo_0
features_0 = geo_0.drop(['id','product'],axis=1)
target_0 = geo_0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size = 0.25, random_state=12345)

print(features_train_0.shape)
print(features_valid_0.shape)
print(target_train_0.shape)
print(target_valid_0.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [12]:
# Standardizing the numeric values
numeric = ['f0','f1','f2']

scaler = StandardScaler()
scaler.fit(features_train_0[numeric])
features_train_0[numeric] = scaler.transform(features_train_0[numeric])
features_valid_0[numeric] = scaler.transform(features_valid_0[numeric])
features_train_0.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


In [13]:
# Creating and testing linear regression model
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = pd.Series(model_0.predict(features_valid_0),index=target_valid_0.index)
mean_predictions_0 = predictions_0.mean() # getting prediction average
mean_target_0 = target_valid_0.mean()
mse_0 = mean_squared_error(predictions_0,target_valid_0) # getting model mean sqaured error
rsme_0 = mse_0 ** 0.5 # Getting root mean squared error
print('Average predicted target:',mean_predictions_0)
print('Region 1 target mean:',mean_target_0)
print('RMSE:',rsme_0)

Average predicted target: 92.59256778438035
Region 1 target mean: 92.07859674082927
RMSE: 37.5794217150813


The average target matches the original dataset's average, which is a good sign but when comparing the model's RMSE it shows the model's predictions aren't very good and is a significant percentage off from the 'baseline'. This indicates the model does well in maintaining the sample's mean but bad at capturing patterns within the dataset. There are likely outliers that are skewing the model's outcomes. Let's get the 99% intervals from the dataset and fit the model again to see if the performance improves.

In [14]:
# Creasting training and validation sets for region 2 model
features_1 = geo_1.drop(['id','product'],axis=1)
target_1 = geo_1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size = 0.25, random_state=12345)

print(features_train_1.shape)
print(features_valid_1.shape)
print(target_train_1.shape)
print(target_valid_1.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [15]:
# standardizing the sets
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])
features_train_1.head()

,f0,f1,f2
27212,-0.850855,0.624428,0.296943
7866,1.971935,1.832275,0.294333
62041,1.079305,0.170127,-0.296418
70185,-1.512028,-0.887837,-0.880471
82230,-1.804775,-0.718311,-0.293255


In [16]:
# Creating region 2 model for testing 
model_1 = LinearRegression()
model_1.fit(features_train_1,target_train_1)
predictions_1 = pd.Series(model_1.predict(features_valid_1),index=target_valid_1.index)
mean_predictions_1 = predictions_1.mean()
mse_1 = mean_squared_error(target_valid_1,predictions_1)
rmse_1 = mse_1 ** 0.5
mean_target_1 = target_1.mean()
print('Region 2 predicted target mean:',mean_predictions_1)
print('Region 2 target average:',mean_target_1)
print('Model RMSE:',rmse_1)

Region 2 predicted target mean: 68.728546895446
Region 2 target average: 68.82500000000002
Model RMSE: 0.893099286775617


When comparing the target and predicted target averages along with this model's rsme value, I can feel confident this model has done a great job identifying patterns within the features. The model's predictions look dependable.

In [17]:
# Creating training and validation sets for region 3 model
features_2 = geo_2.drop(['id','product'],axis=1)
target_2 = geo_2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)
print(features_train_2.shape)
print(features_valid_2.shape)
print(target_train_2.shape)
print(target_valid_2.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [18]:
# Standardizing region 3 features sets
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

features_train_2.head()

,f0,f1,f2
27212,-0.526160,0.776329,-0.400793
7866,-0.889625,-0.404070,-1.222936
62041,-1.133984,0.208576,0.296765
70185,1.227045,1.570166,-0.764556
82230,-0.194289,0.878312,0.840821


In [19]:
# Creating region 3 model for testing
model_2 = LinearRegression()
model_2.fit(features_train_2,target_train_2)
predictions_2 = pd.Series(model_2.predict(features_valid_2),index=target_valid_2.index)
mean_predictions_2 = predictions_2.mean()
mean_target_2 = target_2.mean()
mse_2 = mean_squared_error(target_valid_2,predictions_2)
rmse_2 = mse_2 ** 0.5

print('Region 3 predicted target mean:',mean_predictions_2)
print('Region 3 target average:',mean_target_2)
print('RMSE:',rmse_2)

Region 3 predicted target mean: 94.96504596800489
Region 3 target average: 95.00000000000004
RMSE: 40.02970873393434


Here we see the model for region 3 was spot on in keeping the prediction average on par with the target average. Unfortunately the models rmse score does not give much confidence in the model's predictions as the rmse is about 40% of the target average.

# Profit Calculation
---
Cost for developing 200 wells (`n_wells`) is 100 million USD (`cost`). The target/prediction values are representations of oil reserves. Each unit represents 1000. One barrel of oil is 4.5 USD. We will assign the value of each unit under the `price` variable.

In [20]:
# Assigning key values to variable for calculations
cost = 100000000
price = 4500
n_wells = 200
# Calculating average amount of reserves per well to be profitable
in_profit = cost / n_wells / price
print('Average volume of reserves per well to be in profit is:',in_profit)

Average volume of reserves per well to be in profit is: 111.11111111111111


According to our calculations the development of 200 wells would be profitable around 112 units per well, higher than all three region's average barrels per well.

We'll have to go for the model's highest predicted wells to maximize profits.

In [21]:
# Defining profit calculation function
def profit(predictions, target, n_wells):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:n_wells]
    revenue = 0
    for n in selected:
        revenue += n * price
    profit = (revenue - cost) / 1000000
    return round(profit,ndigits=2)

In [22]:
# Getting top predicted 200 wells for each region
top_region_1 = profit(predictions_0,target_valid_0,n_wells)
top_region_2 = profit(predictions_1,target_valid_1,n_wells)
top_region_3 = profit(predictions_2,target_valid_2,n_wells)
print(f'Top predicted reserves total profit in region 1 is {top_region_1} million USD')
print(f'Top predicted reserves total profit in region 2 is {top_region_2} million USD')
print(f'Top predicted reserves total profit in region 3 is {top_region_3} million USD')

Top predicted reserves total profit in region 1 is 33.21 million USD
Top predicted reserves total profit in region 2 is 24.15 million USD
Top predicted reserves total profit in region 3 is 27.1 million USD


Each regions top 200 wells would yield a great amount of potetial profits but unfortunately there are only funds to get 500 samples for testing to choose the top 200 wells from said samples. We'll incorporate these key values onto our bootstrapping function. 

Using this techinque will get us a normalized profit distribution in order to assess confidence levels.

In [23]:
#  Defining Bootstrapping function to assess confidence levels 
state = np.random.RandomState(12345)
count = 500
def bootstrap(predictions, target, count):
    profit_values = []
    for i in range(1000):
        pred_subsample = predictions.sample(n=count,replace=True,random_state=state)
        target_subsample = target[pred_subsample.index]
        profit_values.append(profit(pred_subsample,target_subsample,n_wells))
    return pd.Series(profit_values)

In [24]:
# Bootstrapping all 3 regions predictions to get a normalized profit distribution
reg_1_profits = bootstrap(predictions_0,target_valid_0,count)
reg_2_profits = bootstrap(predictions_1,target_valid_1,count)
reg_3_profits = bootstrap(predictions_2,target_valid_2,count)

In [25]:
print('REGION 1')
print('Region 1 recorded minimum profit is at a loss for {} million USD\nThe max recorded profit for region 1 is {} million USD'.format(
    reg_1_profits.min(),reg_1_profits.max()))
print('REGION 2')
print('Region 2 recorded minimum profit is at a loss for {} million USD\nThe max recorded profit for region 2 is {} million USD'.format(
    reg_2_profits.min(),reg_2_profits.max()))
print('REGION 3')
print('Region 3 recorded minimum profit is at a loss for {} million USD\nThe max recorded profit for region 3 is {} million USD'.format(
    reg_3_profits.min(),reg_3_profits.max()))

REGION 1
Region 1 recorded minimum profit is at a loss for -3.77 million USD
The max recorded profit for region 1 is 14.1 million USD
REGION 2
Region 2 recorded minimum profit is at a loss for -1.4 million USD
The max recorded profit for region 2 is 11.76 million USD
REGION 3
Region 3 recorded minimum profit is at a loss for -4.38 million USD
The max recorded profit for region 3 is 14.79 million USD


We can see each regions' worst case scenario end with a loss but there is opportunity for huge profits. Let's sure up our confidence in the probabilities in order to make the best business decision possible. 

Let's find the average profits for each region and get a 95% confidence intervals along with risks of loss.

## Risk Assessment
---

In [26]:
# Getting 99% confidence intervals for each regions' distribution
all_profits = [reg_1_profits,reg_2_profits,reg_3_profits]
all_confidence = []
n_count = 0
for region in all_profits:
    mean = region.mean()
    all_confidence.append([region.quantile(0.025),region.quantile(0.975)])
    n_count += 1
    print(f'Region {n_count} average profit is {mean:.2f} million USD\nWe have 95% confidence profits in Region {n_count} will be between {region.quantile(0.025)} million USD and {region.quantile(0.975)} million USD\n')

Region 1 average profit is 4.26 million USD
We have 95% confidence profits in Region 1 will be between -1.0205 million USD and 9.4805 million USD

Region 2 average profit is 5.18 million USD
We have 95% confidence profits in Region 2 will be between 1.28 million USD and 9.540249999999999 million USD

Region 3 average profit is 4.20 million USD
We have 95% confidence profits in Region 3 will be between -1.1609999999999998 million USD and 9.9 million USD



Region 2 looks like a no brainer to me. It appears to be much less likely be at a loss with not much difference in the other regions' top end of possible profit. Let's run the precise numbers of possibility of a loss.

In [27]:
# Calculating risk of loss for each region
reg_count = 0
for region in all_profits:
    risk = sum(region < 0) / 1000
    reg_count += 1
    print(f'The risk of loss for Region {reg_count} is {risk:.2%}')

The risk of loss for Region 1 is 6.00%
The risk of loss for Region 2 is 0.20%
The risk of loss for Region 3 is 6.10%


Region 1 and region 3 have risks of losses at 6% and 6.1% respectively. This exceeds our 2.5% risk of loss. But region 2 has an impressive 0.2% risk. 

## Region Selection/Conclusion
---
Region 2 is our recommendation for region for development. The region has a 0.2% risk of loss and a 95% confidence level profits will be between 1.28 million USD(1.28% ROI) - 9.54 million USD(9.54% ROI.) Region 2 boasts an average of 5.18 million USD profits, at a 5.15% ROI. Out of the 3 regions, region 2 shows the best r/r. It is also the only region that meets the risk of loss threshold.

The model for region 2 is also highly accurate giving us confidence in it's ability to choose the wells with the highest yield. 

In conclusion, region 2 is the only region we'd suggest development for all the calculations and data in this jounal.